In [3]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

# Spark session & context using Spark cluster from docker
spark = spark = SparkSession.builder \
    .master("spark://spark:7077") \
    .appName("Example") \
    .getOrCreate()
# Spark session & context using Spark local PySpark
#spark = SparkSession.builder.master("local[3]").getOrCreate()

sc = spark.sparkContext


Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: java.lang.IllegalArgumentException: requirement failed: Can only call getServletHandlers on a running MetricsSystem
	at scala.Predef$.require(Predef.scala:337)
	at org.apache.spark.metrics.MetricsSystem.getServletHandlers(MetricsSystem.scala:89)
	at org.apache.spark.SparkContext.<init>(SparkContext.scala:650)
	at org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:77)
	at java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
	at java.base/java.lang.reflect.Constructor.newInstanceWithCaller(Constructor.java:499)
	at java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:480)
	at py4j.re

In [2]:
# Sum of the first 100 whole numbers
ds = sc.parallelize(range(100 + 1))
ds.sum()
# 5050

5050

In [3]:
df = spark.read.csv(
    './data/stocks_price_final.csv',
    sep=',',
    header=True)

df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- symbol: string (nullable = true)
 |-- date: string (nullable = true)
 |-- open: string (nullable = true)
 |-- high: string (nullable = true)
 |-- low: string (nullable = true)
 |-- close: string (nullable = true)
 |-- volume: string (nullable = true)
 |-- adjusted: string (nullable = true)
 |-- market.cap: string (nullable = true)
 |-- sector: string (nullable = true)
 |-- industry: string (nullable = true)
 |-- exchange: string (nullable = true)



In [4]:
df.dtypes

[('_c0', 'string'),
 ('symbol', 'string'),
 ('date', 'string'),
 ('open', 'string'),
 ('high', 'string'),
 ('low', 'string'),
 ('close', 'string'),
 ('volume', 'string'),
 ('adjusted', 'string'),
 ('market.cap', 'string'),
 ('sector', 'string'),
 ('industry', 'string'),
 ('exchange', 'string')]

In [5]:
#df.show()
#df.head(10)
#df.tail(2)
#df.first()
df.take(3)

[Row(_c0='1', symbol='TXG', date='2019-09-12', open='54', high='58', low='51', close='52.75', volume='7326300', adjusted='52.75', market.cap='$9.31B', sector='Capital Goods', industry='Biotechnology: Laboratory Analytical Instruments', exchange='NASDAQ'),
 Row(_c0='2', symbol='TXG', date='2019-09-13', open='52.75', high='54.355', low='49.150002', close='52.27', volume='1025200', adjusted='52.27', market.cap='$9.31B', sector='Capital Goods', industry='Biotechnology: Laboratory Analytical Instruments', exchange='NASDAQ'),
 Row(_c0='3', symbol='TXG', date='2019-09-16', open='52.450001', high='56', low='52.009998', close='55.200001', volume='269900', adjusted='55.200001', market.cap='$9.31B', sector='Capital Goods', industry='Biotechnology: Laboratory Analytical Instruments', exchange='NASDAQ')]

In [6]:
#df.describe()

In [7]:
df.columns

['_c0',
 'symbol',
 'date',
 'open',
 'high',
 'low',
 'close',
 'volume',
 'adjusted',
 'market.cap',
 'sector',
 'industry',
 'exchange']

In [8]:
df.count()

1729034

In [9]:
#df.distinct().count()

In [10]:
df.show(3)

+---+------+----------+---------+------+---------+---------+-------+---------+----------+-------------+--------------------+--------+
|_c0|symbol|      date|     open|  high|      low|    close| volume| adjusted|market.cap|       sector|            industry|exchange|
+---+------+----------+---------+------+---------+---------+-------+---------+----------+-------------+--------------------+--------+
|  1|   TXG|2019-09-12|       54|    58|       51|    52.75|7326300|    52.75|    $9.31B|Capital Goods|Biotechnology: La...|  NASDAQ|
|  2|   TXG|2019-09-13|    52.75|54.355|49.150002|    52.27|1025200|    52.27|    $9.31B|Capital Goods|Biotechnology: La...|  NASDAQ|
|  3|   TXG|2019-09-16|52.450001|    56|52.009998|55.200001| 269900|55.200001|    $9.31B|Capital Goods|Biotechnology: La...|  NASDAQ|
+---+------+----------+---------+------+---------+---------+-------+---------+----------+-------------+--------------------+--------+
only showing top 3 rows



In [11]:
df = df.selectExpr("_c0",\
              'open',\
              'high as low',\
              'low as high',\
              'volume as new_volume',\
              'sector',\
              'industry',\
              'date')

In [12]:
df.show(3)

+---+---------+------+---------+----------+-------------+--------------------+----------+
|_c0|     open|   low|     high|new_volume|       sector|            industry|      date|
+---+---------+------+---------+----------+-------------+--------------------+----------+
|  1|       54|    58|       51|   7326300|Capital Goods|Biotechnology: La...|2019-09-12|
|  2|    52.75|54.355|49.150002|   1025200|Capital Goods|Biotechnology: La...|2019-09-13|
|  3|52.450001|    56|52.009998|    269900|Capital Goods|Biotechnology: La...|2019-09-16|
+---+---------+------+---------+----------+-------------+--------------------+----------+
only showing top 3 rows



In [13]:
df.write.csv('./output/stocks_price_final.csv')